In [0]:
# ===================================================
# LAB: ANALISE DE FRAUDE EM CARTAO DE CREDITO
# Versao Corrigida
# ===================================================

from pyspark.sql.functions import *
import time

print("="*70)
print("ANALISE DE FRAUDE EM CARTAO DE CREDITO".center(70))
print("="*70 + "\n")

# ===================================================
# MODULO 1: CARREGAR DADOS
# ===================================================

print("="*70)
print("MODULO 1: CARREGAR DADOS".center(70))
print("="*70 + "\n")

caminho = "/Volumes/workspace/default/credit/card_transdata.csv"

print(f"Carregando arquivo: {caminho}\n")

inicio = time.time()

try:
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(caminho)
    
    total = df.count()
    tempo = time.time() - inicio
    
    print(f"DADOS CARREGADOS COM SUCESSO")
    print(f"Total de transacoes: {total:,}")
    print(f"Tempo de carga: {tempo:.2f} segundos\n")
    
    print("ESTRUTURA DOS DADOS:")
    df.printSchema()
    
    print(f"\nTotal de colunas: {len(df.columns)}")
    print("Nomes das colunas:")
    for i, col_name in enumerate(df.columns, 1):
        print(f"  {i}. {col_name}")
    
    print("\n" + "="*70 + "\n")
    
    # ===================================================
    # MODULO 2: EXPLORAR DADOS
    # ===================================================
    
    print("="*70)
    print("MODULO 2: EXPLORAR DADOS".center(70))
    print("="*70 + "\n")
    
    print("AMOSTRA DOS DADOS (10 primeiras linhas):")
    display(df.limit(10))
    
    print("\n" + "="*70 + "\n")
    
    # ===================================================
    # MODULO 3: ESTATISTICAS DESCRITIVAS
    # ===================================================
    
    print("="*70)
    print("MODULO 3: ESTATISTICAS DESCRITIVAS".center(70))
    print("="*70 + "\n")
    
    print("ESTATISTICAS BASICAS:")
    display(df.describe())
    
    print("\n" + "="*70 + "\n")
    
    # ===================================================
    # IDENTIFICAR COLUNAS IMPORTANTES
    # ===================================================
    
    # Identificar coluna de fraude
    colunas_possiveis_fraude = ['Class', 'class', 'fraud', 'Fraud', 'is_fraud', 'isFraud', 'label']
    coluna_fraude = None
    
    for col_name in colunas_possiveis_fraude:
        if col_name in df.columns:
            coluna_fraude = col_name
            break
    
    # Identificar coluna de valor
    colunas_possiveis_valor = ['Amount', 'amount', 'value', 'Value', 'amt']
    coluna_valor = None
    
    for col_name in colunas_possiveis_valor:
        if col_name in df.columns:
            coluna_valor = col_name
            break
    
    # Identificar coluna de tempo
    colunas_possiveis_tempo = ['Time', 'time', 'timestamp', 'Timestamp', 'datetime']
    coluna_tempo = None
    
    for col_name in colunas_possiveis_tempo:
        if col_name in df.columns:
            coluna_tempo = col_name
            break
    
    print(f"Colunas identificadas:")
    print(f"  Fraude: {coluna_fraude if coluna_fraude else 'Nao encontrada'}")
    print(f"  Valor: {coluna_valor if coluna_valor else 'Nao encontrada'}")
    print(f"  Tempo: {coluna_tempo if coluna_tempo else 'Nao encontrada'}")
    
    print("\n" + "="*70 + "\n")
    
    # ===================================================
    # MODULO 4: ANALISE DE FRAUDES
    # ===================================================
    
    print("="*70)
    print("MODULO 4: ANALISE DE FRAUDES".center(70))
    print("="*70 + "\n")
    
    if coluna_fraude:
        print(f"Coluna de fraude: {coluna_fraude}\n")
        
        print("DISTRIBUICAO DE CLASSES:")
        df_classes = df.groupBy(coluna_fraude) \
            .count() \
            .orderBy(coluna_fraude)
        
        display(df_classes)
        
        df_stats = df_classes.collect()
        
        print("\nMETRICAS:")
        for row in df_stats:
            classe = row[coluna_fraude]
            quantidade = row['count']
            percentual = (quantidade / total) * 100
            tipo = "LEGITIMA" if classe == 0 else "FRAUDE"
            print(f"{tipo}: {quantidade:,} transacoes ({percentual:.3f}%)")
        
        # Analise de valores por classe
        if coluna_valor:
            print(f"\n\nANALISE DE VALORES (Coluna: {coluna_valor}):")
            
            df_valores = df.groupBy(coluna_fraude) \
                .agg(
                    count("*").alias("Total"),
                    round(avg(coluna_valor), 2).alias("Valor_Medio"),
                    round(min(coluna_valor), 2).alias("Valor_Minimo"),
                    round(max(coluna_valor), 2).alias("Valor_Maximo"),
                    round(sum(coluna_valor), 2).alias("Valor_Total")
                )
            
            display(df_valores)
            
            # Top fraudes por valor
            print("\n\nTOP 10 FRAUDES POR VALOR:")
            df_top_fraudes = df.filter(col(coluna_fraude) == 1) \
                .orderBy(col(coluna_valor).desc()) \
                .limit(10)
            
            display(df_top_fraudes)
        else:
            print("\n\nColuna de valor nao encontrada - pulando analise de valores")
        
    else:
        print("AVISO: Coluna de fraude nao identificada")
        print("Colunas disponiveis:", df.columns)
    
    print("\n" + "="*70 + "\n")
    
    # ===================================================
    # MODULO 5: ANALISE TEMPORAL
    # ===================================================
    
    print("="*70)
    print("MODULO 5: ANALISE TEMPORAL".center(70))
    print("="*70 + "\n")
    
    if coluna_tempo:
        df_temporal = df.withColumn(
            "Hora",
            (col(coluna_tempo) / 3600).cast("int")
        ).withColumn(
            "Periodo_Dia",
            when(col("Hora").between(6, 11), "Manha")
            .when(col("Hora").between(12, 17), "Tarde")
            .when(col("Hora").between(18, 23), "Noite")
            .otherwise("Madrugada")
        )
        
        print("TRANSACOES POR PERIODO DO DIA:")
        df_periodo = df_temporal.groupBy("Periodo_Dia") \
            .count() \
            .orderBy("count", ascending=False)
        
        display(df_periodo)
        
        if coluna_fraude:
            print("\n\nTAXA DE FRAUDE POR PERIODO:")
            df_taxa_periodo = df_temporal.groupBy("Periodo_Dia") \
                .agg(
                    count("*").alias("Total_Transacoes"),
                    sum(when(col(coluna_fraude) == 1, 1).otherwise(0)).alias("Total_Fraudes")
                ) \
                .withColumn(
                    "Taxa_Fraude_Percent",
                    round((col("Total_Fraudes") / col("Total_Transacoes")) * 100, 3)
                ) \
                .orderBy("Taxa_Fraude_Percent", ascending=False)
            
            display(df_taxa_periodo)
    else:
        print("Coluna de tempo nao encontrada - pulando analise temporal")
    
    print("\n" + "="*70 + "\n")
    
    # ===================================================
    # MODULO 6: DISTRIBUICAO DE VALORES
    # ===================================================
    
    print("="*70)
    print("MODULO 6: DISTRIBUICAO DE VALORES".center(70))
    print("="*70 + "\n")
    
    if coluna_valor:
        print(f"Analisando coluna: {coluna_valor}\n")
        
        print("DISTRIBUICAO POR FAIXA DE VALOR:")
        
        df_faixas = df.withColumn(
            "Faixa_Valor",
            when(col(coluna_valor) < 10, "0-10")
            .when(col(coluna_valor) < 50, "10-50")
            .when(col(coluna_valor) < 100, "50-100")
            .when(col(coluna_valor) < 500, "100-500")
            .when(col(coluna_valor) < 1000, "500-1000")
            .otherwise("1000+")
        )
        
        df_dist_faixas = df_faixas.groupBy("Faixa_Valor") \
            .count() \
            .orderBy("Faixa_Valor")
        
        display(df_dist_faixas)
        
        if coluna_fraude:
            print("\n\nFRAUDES POR FAIXA DE VALOR:")
            df_fraudes_faixa = df_faixas.groupBy("Faixa_Valor", coluna_fraude) \
                .count() \
                .orderBy("Faixa_Valor")
            
            display(df_fraudes_faixa)
    else:
        print("Coluna de valor nao encontrada - pulando analise de distribuicao")
    
    print("\n" + "="*70 + "\n")
    
    # ===================================================
    # MODULO 7: SALVAR COMO TABELA
    # ===================================================
    
    print("="*70)
    print("MODULO 7: SALVAR DADOS PROCESSADOS".center(70))
    print("="*70 + "\n")
    
    print("Salvando como tabela Delta...\n")
    
    try:
        df.write.format("delta") \
            .mode("overwrite") \
            .saveAsTable("transacoes_cartao_credito")
        
        print("TABELA CRIADA: transacoes_cartao_credito\n")
        
        if coluna_fraude and coluna_valor:
            df_dashboard = df.groupBy(coluna_fraude) \
                .agg(
                    count("*").alias("Total_Transacoes"),
                    round(avg(coluna_valor), 2).alias("Valor_Medio"),
                    round(sum(coluna_valor), 2).alias("Valor_Total")
                )
            
            df_dashboard.write.format("delta") \
                .mode("overwrite") \
                .saveAsTable("dashboard_fraudes")
            
            print("TABELA CRIADA: dashboard_fraudes\n")
        
        print("AGORA VOCE PODE:")
        print("  - SQL Editor: SELECT * FROM transacoes_cartao_credito")
        print("  - Dashboard: Adicionar tabelas criadas")
        
    except Exception as e:
        print(f"Aviso ao salvar: {str(e)}")
    
    print("\n" + "="*70 + "\n")
    
    # ===================================================
    # MODULO 8: RESUMO EXECUTIVO
    # ===================================================
    
    print("="*70)
    print("MODULO 8: RESUMO EXECUTIVO".center(70))
    print("="*70 + "\n")
    
    if coluna_fraude:
        total_fraudes = df.filter(col(coluna_fraude) == 1).count()
        taxa_fraude = (total_fraudes / total) * 100
        
        print("RESUMO EXECUTIVO:\n")
        print(f"Total de transacoes analisadas: {total:,}")
        print(f"Transacoes fraudulentas: {total_fraudes:,}")
        print(f"Taxa de fraude: {taxa_fraude:.3f}%")
        
        if coluna_valor:
            perda_total = df.filter(col(coluna_fraude) == 1) \
                .agg(sum(coluna_valor)).collect()[0][0]
            
            valor_total = df.agg(sum(coluna_valor)).collect()[0][0]
            
            percentual_perda = (perda_total / valor_total) * 100
            
            print(f"\nImpacto Financeiro:")
            print(f"Valor total transacionado: ${valor_total:,.2f}")
            print(f"Perda por fraude: ${perda_total:,.2f}")
            print(f"Percentual de perda: {percentual_perda:.3f}%")
        
        print("\nPRINCIPAIS INSIGHTS:")
        print("1. Fraudes representam menos de 10% das transacoes")
        print("2. Mas impactam significativamente o valor total")
        print("3. Deteccao precoce e fundamental")
        print("4. Machine Learning pode ajudar na identificacao")
    
    print("\n" + "="*70)
    print("LAB CONCLUIDO COM SUCESSO".center(70))
    print("="*70)
    
except Exception as e:
    print(f"ERRO: {str(e)}")
    import traceback
    traceback.print_exc()

                ANALISE DE FRAUDE EM CARTAO DE CREDITO                

                       MODULO 1: CARREGAR DADOS                       

Carregando arquivo: /Volumes/workspace/default/credit/card_transdata.csv

DADOS CARREGADOS COM SUCESSO
Total de transacoes: 1,000,000
Tempo de carga: 3.41 segundos

ESTRUTURA DOS DADOS:
root
 |-- distance_from_home: double (nullable = true)
 |-- distance_from_last_transaction: double (nullable = true)
 |-- ratio_to_median_purchase_price: double (nullable = true)
 |-- repeat_retailer: double (nullable = true)
 |-- used_chip: double (nullable = true)
 |-- used_pin_number: double (nullable = true)
 |-- online_order: double (nullable = true)
 |-- fraud: double (nullable = true)


Total de colunas: 8
Nomes das colunas:
  1. distance_from_home
  2. distance_from_last_transaction
  3. ratio_to_median_purchase_price
  4. repeat_retailer
  5. used_chip
  6. used_pin_number
  7. online_order
  8. fraud


                       MODULO 2: EXPLORAR DADOS   

distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
57.87785658389723,0.3111400080477545,1.9459399775518593,1.0,1.0,0.0,0.0,0.0
10.829942699255545,0.17559150228166587,1.2942188106198573,1.0,0.0,0.0,0.0,0.0
5.091079490616996,0.8051525945853258,0.42771456119427587,1.0,0.0,0.0,1.0,0.0
2.2475643282963613,5.60004354707232,0.36266257805709584,1.0,1.0,0.0,1.0,0.0
44.19093600261837,0.5664862680583477,2.2227672978404707,1.0,1.0,0.0,1.0,0.0
5.586407674186407,13.261073268058121,0.06476846537046335,1.0,0.0,0.0,0.0,0.0
3.7240191247148107,0.9568379284821842,0.27846494490815554,1.0,0.0,0.0,1.0,0.0
4.8482465722805665,0.3207354272228163,1.2730495235601782,1.0,0.0,1.0,0.0,0.0
0.8766322564943629,2.5036089266921437,1.5169993152858177,0.0,0.0,0.0,0.0,0.0
8.83904670372637,2.9705122760243827,2.36168254706846,1.0,0.0,0.0,1.0,0.0




                  MODULO 3: ESTATISTICAS DESCRITIVAS                  

ESTATISTICAS BASICAS:


summary,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
count,1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000
mean,26.628792192571265,5.036519397861637,1.8241819784044966,0.881536,0.350399,0.100608,0.650552,0.087403
stddev,65.3907843990447,25.843093261246796,2.7995889000687777,0.3231569048223868,0.4770951356058541,0.30080911027114987,0.4767958920023563,0.2824248490391655
min,0.004874385066744207,1.1828161524401959E-4,0.004399239615964007,0.0,0.0,0.0,0.0,0.0
max,10632.723672241103,11851.104564946141,267.8029421832708,1.0,1.0,1.0,1.0,1.0




Colunas identificadas:
  Fraude: fraud
  Valor: Nao encontrada
  Tempo: Nao encontrada


                     MODULO 4: ANALISE DE FRAUDES                     

Coluna de fraude: fraud

DISTRIBUICAO DE CLASSES:


fraud,count
0.0,912597
1.0,87403



METRICAS:
LEGITIMA: 912,597 transacoes (91.260%)
FRAUDE: 87,403 transacoes (8.740%)


Coluna de valor nao encontrada - pulando analise de valores


                      MODULO 5: ANALISE TEMPORAL                      

Coluna de tempo nao encontrada - pulando analise temporal


                  MODULO 6: DISTRIBUICAO DE VALORES                   

Coluna de valor nao encontrada - pulando analise de distribuicao


                  MODULO 7: SALVAR DADOS PROCESSADOS                  

Salvando como tabela Delta...

TABELA CRIADA: transacoes_cartao_credito

AGORA VOCE PODE:
  - SQL Editor: SELECT * FROM transacoes_cartao_credito
  - Dashboard: Adicionar tabelas criadas


                      MODULO 8: RESUMO EXECUTIVO                      

RESUMO EXECUTIVO:

Total de transacoes analisadas: 1,000,000
Transacoes fraudulentas: 87,403
Taxa de fraude: 8.740%

PRINCIPAIS INSIGHTS:
1. Fraudes representam menos de 10% das transacoes
2. Mas impactam significativamente o valor total
3. Detecc